In [1]:
from pyspark.sql import SparkSession

# Criar uma SparkSession com as configurações para MinIO
spark = (
    SparkSession
    .builder.appName("PySpark_Minio_Demo")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.901") # dependencias do minio
    .config("spark.hadoop.fs.s3a.endpoint", "http://dlkdataway-hl.storage:9000") # caminho do service do minio no kubernetes
    .config("spark.hadoop.fs.s3a.access.key", "WZOjGHQ2PMFQAixY7VzO")
    .config("spark.hadoop.fs.s3a.secret.key", "39OTdBH94gJ3E9pqQCpQxDDdOcYJzykf46Ry70nZ")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.mapreduce.job.metrics.exclude-types", "*") # Evita o warning da não localização do hadoop-metrics
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") # Evita o warning Using standard FileOutputCommitter
    .config("spark.hadoop.mapreduce.outputcommitter.factory.scheme.s3a", "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory") # Evita o warning Using standard FileOutputCommitter
    .config("spark.hadoop.fs.s3a.committer.name", "directory") # ou 'partitioned' dependendo do caso
    .config("spark.hadoop.fs.s3a.committer.magic.enabled", "true") # Habilite 'true' se estiver usando magic committer
    .getOrCreate()
)

:: loading settings :: url = jar:file:/app/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-33c19e77-6ea1-4dd2-a7de-6c9c066ee78b;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 399ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|d

In [2]:
# Exemplo de dados
data = [("John", 28), ("Anna", 23), ("Tom", 35)]
columns = ["Nome", "Idade"]

# Criar DataFrame a partir dos dados
df = spark.createDataFrame(data, columns)

# Caminho para o bucket do MinIO (usando a sintaxe s3a://)
bucket_path = "s3a://stage-sandbox01/teste2.csv"

In [3]:
# Gravar o DataFrame como CSV no MinIO
df.write.csv(bucket_path, mode="overwrite", header=True)

24/10/09 03:52:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/10/09 03:52:30 WARN AbstractS3ACommitter: Task committer attempt_20241009035223576710473006160397_0000_m_000000_0: No pending uploads to commit


In [ ]:
# Parar a sessão do Spark
spark.stop()